In [1]:
import tensorflow as tf

/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import pandas as pd
from autograd import numpy as np

In [5]:
df_train = pd.read_csv("./traindata.csv", dtype=np.uint8)

In [6]:
df_train.head(3)

,id,0,1,2,3,4,5,6,7,8,...,775,776,777,778,779,780,781,782,783,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [7]:
df_train.shape

(60000, 786)

In [8]:
df_imgs = df_train.drop(['label', 'id'], axis=1)
x_original = df_imgs.as_matrix().T

print("x_original shape: ", x_original.shape)

x_original shape:  (784, 60000)


In [9]:
y_original = df_train['label'].as_matrix()

print("y_original shape: ", y_original.shape)

y_original shape:  (60000,)


In [12]:
num_sample = 5000
inds = np.random.permutation(y_original.shape[0])[:num_sample]
x_sample = x_original[:,inds]
y_sample = y_original[inds]

In [13]:
print("x_sample shape: ", x_sample.shape)
print("y_sample shape: ", y_sample.shape)

x_sample shape:  (784, 5000)
y_sample shape:  (5000,)


In [44]:
x = x_original
y = y_original

In [45]:
xmax = np.max(x)
x_scale = x/xmax

In [46]:
input_dim = x_scale.shape[1]
nb_classes = y.shape[0]

print("input_dim: ", input_dim)
print("nb_classes: ", nb_classes)

input_dim:  784
nb_classes:  5000


In [123]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

In [124]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

In [125]:
model.fit(x_scale, y, epochs=5, verbose=2)

Epoch 1/5
 - 2s - loss: 0.5536 - acc: 0.8396
Epoch 2/5
 - 1s - loss: 0.2437 - acc: 0.9330
Epoch 3/5
 - 1s - loss: 0.1665 - acc: 0.9552
Epoch 4/5
 - 1s - loss: 0.1084 - acc: 0.9712
Epoch 5/5
 - 1s - loss: 0.0752 - acc: 0.9784


In [126]:
df_testdata = pd.read_csv("./testdata.csv", dtype=np.uint8)
df_testdata.head(3)

,id,0,1,2,3,4,5,6,7,8,...,775,776,777,778,779,780,781,782,783,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [127]:
df_testimgs = df_testdata.drop(['label', 'id'], axis=1)
xtest = df_testimgs.as_matrix()

print("xtest shape:", xtest.shape)

xtest shape: (10000, 784)


In [128]:
ytest = df_testdata['label'].as_matrix()
# ytest = ytest.reshape((10000, 1)).T
print("ytest shape:", ytest.shape)

ytest shape: (10000,)


In [129]:
xtest_scale = xtest/xmax

In [130]:
model.evaluate(xtest_scale, ytest)

10000/10000 [==============================] - 1s 99us/step


[0.18678534353934229, 0.9443]

In [119]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(input_dim, 
                                input_dim=input_dim, 
                                activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(nb_classes, activation=tf.nn.softmax))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

In [121]:
model.fit(x_scale, y, epochs=10, verbose=2)

Epoch 1/10
 - 11s - loss: 0.0855 - acc: 0.9776
Epoch 2/10
 - 11s - loss: 0.0538 - acc: 0.9892
Epoch 3/10
 - 11s - loss: 0.0396 - acc: 0.9924
Epoch 4/10
 - 11s - loss: 0.0256 - acc: 0.9964
Epoch 5/10
 - 11s - loss: 0.0189 - acc: 0.9990
Epoch 6/10
 - 11s - loss: 0.0124 - acc: 0.9994
Epoch 7/10
 - 11s - loss: 0.0094 - acc: 0.9992
Epoch 8/10
 - 11s - loss: 0.0068 - acc: 0.9998
Epoch 9/10
 - 11s - loss: 0.0052 - acc: 1.0000
Epoch 10/10
 - 11s - loss: 0.0043 - acc: 1.0000


In [122]:
model.evaluate(xtest_scale, ytest)

10000/10000 [==============================] - 4s 431us/step


[0.19767896809149532, 0.9525]

aaa

In [135]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(input_dim, 
                                input_dim=input_dim, 
                                activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(input_dim,
                                activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(input_dim,
                                activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(nb_classes, activation=tf.nn.softmax))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

In [136]:
model.fit(x_scale, y, epochs=20, verbose=2)

Epoch 1/10
 - 16s - loss: 0.8755 - acc: 0.7748
Epoch 2/10
 - 13s - loss: 0.2174 - acc: 0.9352
Epoch 3/10
 - 13s - loss: 0.1445 - acc: 0.9548
Epoch 4/10
 - 13s - loss: 0.0902 - acc: 0.9726
Epoch 5/10
 - 13s - loss: 0.0692 - acc: 0.9776
Epoch 6/10
 - 13s - loss: 0.0419 - acc: 0.9868
Epoch 7/10
 - 13s - loss: 0.0312 - acc: 0.9898
Epoch 8/10
 - 13s - loss: 0.0558 - acc: 0.9834
Epoch 9/10
 - 13s - loss: 0.0421 - acc: 0.9890
Epoch 10/10
 - 14s - loss: 0.0468 - acc: 0.9880


In [137]:
model.evaluate(xtest_scale, ytest)

10000/10000 [==============================] - 5s 477us/step


[0.27374397583268584, 0.9426]